In [69]:
import requests
import urllib
import html2text as ht
import re
import pandas as pd
import numpy as np
import json
import string
from bs4 import BeautifulSoup
from collections import Counter

## Scrape Billboard for song titles

In [460]:
class DateTracker:
    
    days = {1: 31, 2: 28, 3: 31, 4: 30, 5: 31, 6: 30, 
            7: 31, 8: 31, 9: 30, 10: 31, 11: 30, 12: 31}
    
    def __init__(self, year = None, month = None, day = None):
        self.year = year
        self.month = month
        self.day = day
        
    def previous_week(self):
        
        if (self.year == 1958) and (self.month == 8) and (0 < self.day <= 7):
            return
        
        # if move to previous month
        if self.day - 7 <= 0:
            
            if self.month != 1:
                
                self.month -= 1
                self.day += self.days[self.month] - 7
            
            # if move to previous year
            else:
                
                self.year -= 1
                self.month = 12
                self.day += self.days[self.month] - 7
                
        else:
            
            self.day -= 7
            
    def as_date(self):
        
        return "%04d-%02d-%02d" % (self.year, self.month, self.day)
        
        
class Song:
    
    def __init__(self, artist, name, rank, peakpos, weeksonchart, date):
        self.artist = artist
        self.name = name
        self.rank = rank
        self.peakpos = peakpos
        self.weeksonchart = weeksonchart
        self.date = date
        
    def get_artist(self):
        return self.artist
    
    def get_name(self):
        return self.name
    
    def get_rank(self):
        return self.rank
    
    def get_peak_pos(self):
        return self.peakpos
    
    def get_weeks_on_chart(self):
        return self.weeksonchart
    
    def get_date(self):
        return self.date
    
    def list_form(self):
        return [self.artist, self.name, self.rank, self.peakpos,
                self.weeksonchart, self.date]
    
    def summary(self):
        print("\n".join([str(x) for x in [self.artist, self.name,
                                          self.rank, self.peakpos,
                                          self.weeksonchart, self.date]]))

In [461]:
def clean_text(text):
    bad = {"\\\\n": "\n", 
           "\n+": "\n", 
           "\n": " - ",
           "\[\]\(.{0,}?\)": "",
           "\[Play\]\(.{0,}?\)": "",
           "\[\s{0,5}-\s{0,5}Song\s{0,5}-\s{0,5}Lyrics\s{0,5}-\s{0,5}\]\(.{0,}?\)": ""}

    for x in bad:
        text = re.sub(x, bad[x], text, flags = re.I | re.S)
        
    return text

def clean_line(text):
    repl = {"weeks at no. 1": "",
            "(- !)+": "",
            "(?<![A-z])! -": "",
            "_": "",
            "\[\s-": "", 
            "-\s\]": "",
            "\[.{0,}?\]|\(.{0,}?\)": "",
            "\s+": " ",
            "(-\s{0,5})+": "- ",}    
    
    for x in repl:
        text = re.sub(x, repl[x], text, flags = re.I | re.S)
    return text

In [462]:
def get_songs_by_week(weeks = 100):
    
    filler = [" and ", " featuring ", " & ", " x ", " / "]
    path = "https://www.billboard.com/charts/hot-100"
    curdate = DateTracker(year = 2018, month = 8, day = 11)
    songs = []

    for j in range(weeks + 1):
        
        page = requests.get(path + "/" + curdate.as_date())
        if j % 20 == 0:
            print("We are on week {}, it is {}".format(j, curdate.as_date()))

        data = ht.html2text(str(page.content))
        data = clean_text(data)
        data = re.split("date search |in performance - |on chart - ", data, flags = re.I)[1:]

        #first one
        weeksoc = int(re.search("[0-9]{0,1}\s-\sweeks", 
                                re.sub("weeks at no. 1", "", data[0], flags = re.I | re.S),
                                flags = re.I | re.S).group()[:2])
        tmp = clean_line(data[1]).strip().split("-")
        name = tmp[0].strip()
        artist = tmp[1].strip()
        for k in filler:
            artist = re.sub(k, ", ", artist, flags = re.I | re.S)

        songs.append(Song(artist = artist, name = name, rank = 1,
                          peakpos = 1, weeksonchart = weeksoc, 
                          date = curdate.as_date()))


        #everything else (2 - 100)
        for i in data[2:]:
            #we know we have a rank
            if re.match("^[0-9]{0,3}", i.strip()).group() != "":
                i = clean_line(i)
                tmp = i.split("-")
                rank = tmp[0].strip()
                name = tmp[1].strip()
                artist = tmp[2].strip()
                for k in filler:
                    artist = re.sub(k, ", ", artist, flags = re.I | re.S)
                    
                #if these are empty they're debuts!
                try:
                    peakpos = int(re.search("[0-9]{0,3}\s-\speak position", i, flags = re.I | re.S).group().split(" ")[0])
                except AttributeError:
                    peakpos = ""
                try:
                    weeksoc = int(re.search("[0-9]{0,3}\s-\sweeks", i, flags = re.I | re.S).group().split(" ")[0])
                except AttributeError:
                    weeksoc = ""

                songs.append(Song(artist = artist, name = name, rank = rank,
                                  peakpos = peakpos, weeksonchart = weeksoc,
                                  date = curdate.as_date()))

        curdate.previous_week()
        
    return songs, curdate

In [463]:
songs, curdate = get_songs_by_week(weeks = 10)

We are on week 0, it is 2018-08-11


In [467]:
prep = np.array([x.list_form() for x in songs])
df = pd.DataFrame({
    "Artists": prep[:, 0],
    "Name": prep[:, 1],
    "Weekly rank": prep[:, 2],
    "Peak position": prep[:, 3],
    "Weeks on chart": prep[:, 4],
    "Week": prep[:, 5]
})

In [ ]:
df.groupby("Artists")[["Peak position"]].max()

In [458]:
lol = 0
for i in songs:
    i.summary()
    print()
    lol+=1
    if lol == 2000:
        break

Drake
In My Feelings
1
1
5
2018-08-11

6ix9ine, Nicki Minaj, Murda Beatz
FEFE
3
3
2
2018-08-11

Maroon 5, Cardi B
Girls Like You
4
3
10
2018-08-11

DJ Khaled, Justin Bieber, Chance The Rapper, Quavo
No Brainer
5


2018-08-11

Post Malone
Better Now
6
5
14
2018-08-11

Juice WRLD
Lucid Dreams
7
3
12
2018-08-11

Ella Mai
Boo'd Up
8
5
18
2018-08-11

Tyga, Offset
Taste
9
9
10
2018-08-11

Drake
Nice For What
10
1
17
2018-08-11

Post Malone, Ty Dolla $ign
Psycho
11
1
23
2018-08-11

Ariana Grande
No Tears Left To Cry
12
3
15
2018-08-11

Taylor Swift
Delicate
13
12
21
2018-08-11

Lil Baby, Drake
Yes Indeed
14
6
12
2018-08-11

Khalid, Normani
Love Lies
15
15
24
2018-08-11

Drake
God's Plan
16
1
28
2018-08-11

Zedd, Maren Morris, Grey
The Middle
17
5
27
2018-08-11

Ed Sheeran
Perfect
18
1
49
2018-08-11

Bebe Rexha, Florida Georgia Line
Meant To Be
19
2
41
2018-08-11

Marshmello, Anne
Friends
20
11
25
2018-08-11

XXXTENTACION
Sad!
21
1
22
2018-08-11

Kenny Chesney
Get Along
22
22
17
2018-08-11

5 